In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, LSTM
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Embedding, Dropout
from keras.models import Model
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import optimizers

In [ ]:
classes = ['target', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack']
data = pd.read_csv('/content/drive/MyDrive/Untitled folder/train.csv').fillna(' ')
data['comment_text'].fillna("missing", inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data["comment_text"],data[classes],test_size=0.15,random_state=76)
data_text = pd.concat([X_train,X_test])
print(type(X_train))

<class 'pandas.core.series.Series'>


In [ ]:
data

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:41.987077+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:42.870083+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:45.222647+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:47.601894+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0,0,0,0,0,0,0,0,0,0,0.25,0,0,0,0,0,0,0,0,0,0,0,0,0,2015-09-29 10:50:48.488476+00,2,,2006,rejected,0,0,0,1,0,0.0,4,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804869,6333967,0.000000,"Maybe the tax on ""things"" would be collected w...",0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 00:55:39.265997+00,54,6.33238e+06,399385,approved,0,0,0,0,0,0.0,0,4
1804870,6333969,0.000000,What do you call people who STILL think the di...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 00:55:56.634498+00,54,6.33387e+06,399528,approved,0,0,0,0,0,0.0,0,4
1804871,6333982,0.000000,"thank you ,,,right or wrong,,, i am following ...",0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 00:59:02.840713+00,54,,399457,approved,0,0,0,0,0,0.0,0,4
1804872,6334009,0.621212,Anyone who is quoted as having the following e...,0.030303,0.030303,0.045455,0.621212,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 01:01:10.153692+00,13,6.33383e+06,399519,approved,0,0,0,0,0,0.0,0,66


In [ ]:
#tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(data_text))
X_traint = tokenizer.texts_to_sequences(X_train)
X_testt = tokenizer.texts_to_sequences(X_test)

In [ ]:
#padding 
X_trainp = pad_sequences(X_traint, maxlen=250)
X_testp = pad_sequences(X_testt, maxlen=250)

In [ ]:
#glove embedding
GLOVE_6B_50D_PATH = "/content/drive/MyDrive/Untitled folder/glove.6B.50d.txt"
index_len = len(tokenizer.word_index)
glove = {}
dimension = 50
glove_matrix = np.zeros((index_len+1,dimension))
glove_path = open(GLOVE_6B_50D_PATH)
for line in tqdm(glove_path):
    line_arr = line.split()
    word = line_arr[0]
    glove[word] = np.asarray(line_arr[1:], dtype='float32')
glove_path.close()
for word, idx in tqdm(tokenizer.word_index.items()):
    if word in glove: glove_matrix[idx] = glove[word]

400000it [00:08, 44661.78it/s]
100%|██████████| 397708/397708 [00:00<00:00, 610620.02it/s]


In [ ]:
#lstm model
model_len = 250
model = Sequential()
model.add(Embedding(index_len + 1, dimension, weights = [glove_matrix], input_length = model_len, trainable = False))
model.add(Bidirectional(LSTM(100, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.15))
model.add(Dense(70, activation = "relu"))
model.add(Dropout(0.15))
model.add(Dense(35, activation = "relu"))
model.add(Dropout(0.15))
model.add(Dense(6, activation = 'sigmoid'))
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.Adam(lr = 0.01), metrics = ['accuracy'])
lstm = model.fit(X_trainp, y_train, epochs = 2, batch_size = 150, validation_split = 0.25)
y_pred = model.predict(X_testp, verbose = 3, batch_size = 150)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 50)           19885450  
_________________________________________________________________
bidirectional (Bidirectional (None, 250, 200)          120800    
_________________________________________________________________
global_max_pooling1d (Global (None, 200)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0